In [3]:
#from laspy.file import File
# #import laspy as lp
# #gpd.options.use_pygeos = True
# import pyvista as pv
# import pptk

In [6]:
#!/usr/bin/python3
import numpy as np
# import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import open3d as o3d
from shapely.geometry import Point
from scipy.spatial import Delaunay
import math
from functools import reduce

In [10]:
# read point cloud
data_folder = '/Users/philipp/Projects/PycharmProjects/point-cloud-volume/input/'
dataset = "points-32638.xyz"
npcloud = np.loadtxt(data_folder + dataset, delimiter=',')

crs = None
geometry = [Point(xyz) for xyz in npcloud]
geodf = gpd.GeoDataFrame(npcloud, crs=crs, geometry=geometry)
geodf.crs = {'init': 'epsg:32638'}
# geodf = geodf.to_crs("EPSG:4326")

/Users/philipp/Projects/PycharmProjects/point-cloud-volume/venv/lib/python3.9/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [0]:
# polygon vector
annotations = gpd.read_file('input/site.geojson')
annotations = annotations.to_crs("EPSG:32638")

In [29]:
annotations

,fid,Type,cut,fill,volume,Area2,Typical sp,DDVolume,SiteVolu2,SiteArea2,MaxHeight,SiteCut,SiteFill,geometry
0,3269.0,natural_Soil,3.285872132,-1.45880725,4.744679382,32.402874,yes,NaN,7.80,25.19,1.40,12.7,4.90,"MULTIPOLYGON (((665839.698 2716029.889, 665837..."
1,3232.0,natural_Soil,2.508158,-1.074423,3.582581,25.747415,yes,NaN,9.10,26.64,1.10,13.1,4.00,"MULTIPOLYGON (((665662.619 2715791.778, 665659..."
2,2898.0,natural_Soil,2.463113,-3.087354,-0.624241,25.165588,yes,6.32,4.00,17.75,0.88,8.6,4.50,"MULTIPOLYGON (((665093.602 2715633.487, 665093..."
3,621.0,natural_Soil,28.510577,-2.918489,25.592088,120.926996,yes,48.17,48.80,127.65,1.30,91.1,42.30,"MULTIPOLYGON (((664513.389 2716032.109, 664515..."
4,622.0,natural_Soil,41.785227,-4.413772,37.371455,138.734473,yes,63.57,71.50,109.86,1.50,110.6,39.10,"MULTIPOLYGON (((664531.711 2716037.334, 664534..."
5,623.0,natural_Soil,31.346528,-1.945509,29.401019,117.997395,yes,30.24,56.20,115.45,1.30,111.6,55.40,"MULTIPOLYGON (((664492.288 2716030.786, 664493..."
6,624.0,natural_Soil,19.615540,-0.960564,18.654975,102.200577,yes,25.35,46.50,95.82,1.10,63.9,17.50,"MULTIPOLYGON (((664467.550 2716035.681, 664469..."
7,1751.0,natural_Soil,4.821274,-2.740178,2.081096,33.639115,yes,13.70,9.80,37.75,1.40,24.1,14.30,"MULTIPOLYGON (((663560.370 2716082.914, 663559..."
8,680.0,natural_Soil,2.9,-0.8,2,40.000623,yes,3.23,3.30,12.10,0.78,4.1,0.83,"MULTIPOLYGON (((663498.105 2716111.071, 663500..."
9,1754.0,Construction_Waste,2.322964,-2.680375,-0.357411,31.707809,yes,9.78,4.00,25.91,1.10,16.1,12.10,"MULTIPOLYGON (((663560.370 2716082.914, 663561..."


In [28]:
# Points in polygon
pointInPolys = gpd.tools.sjoin(geodf, annotations, predicate="within", how='left')
pointInPolysNA = pointInPolys.dropna(subset=['fid']).to_numpy()

In [26]:
my_dict = {"fid": [], "volume": [], "points": []}

for i in np.unique(pointInPolysNA[:,9]):
    # print(i)
    subset = pointInPolysNA[pointInPolysNA[:,9] == i, ]
    subsetBase = np.copy(subset)
    subsetBase[:,2]  = min(subset[:,2])
    base = np.append(subset, subsetBase, axis=0)
    baseAr = [tuple(row) for row in base[:,0:3]]
    baseAr = np.unique(baseAr, axis=0)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(baseAr[:, :3])
    pcd.estimate_normals()
    pcd.compute_convex_hull()
    try:
        ch = pcd.compute_convex_hull()
        chv = ch[0].get_volume()
    except RuntimeError:
        # ch = 0
        chv = 0

    my_dict["fid"].append(i)
    my_dict["volume"].append(chv)
    my_dict["points"].append(baseAr.shape[0])

merged = annotations.merge(pd.DataFrame.from_dict(my_dict), on='fid')
# merged.to_file('dataframe1.shp')

In [27]:
merged

,fid,Type,cut,fill,volume_x,Area2,Typical sp,DDVolume,SiteVolu2,SiteArea2,MaxHeight,SiteCut,SiteFill,geometry,volume_y,points
0,2898.0,natural_Soil,2.463113,-3.087354,-0.624241,25.165588,yes,6.32,4.0,17.75,0.88,8.6,4.5,"MULTIPOLYGON (((665093.602 2715633.487, 665093...",0.000000,115
1,621.0,natural_Soil,28.510577,-2.918489,25.592088,120.926996,yes,48.17,48.8,127.65,1.30,91.1,42.3,"MULTIPOLYGON (((664513.389 2716032.109, 664515...",76.758535,558
2,622.0,natural_Soil,41.785227,-4.413772,37.371455,138.734473,yes,63.57,71.5,109.86,1.50,110.6,39.1,"MULTIPOLYGON (((664531.711 2716037.334, 664534...",121.783699,641
3,623.0,natural_Soil,31.346528,-1.945509,29.401019,117.997395,yes,30.24,56.2,115.45,1.30,111.6,55.4,"MULTIPOLYGON (((664492.288 2716030.786, 664493...",0.000000,547
4,624.0,natural_Soil,19.615540,-0.960564,18.654975,102.200577,yes,25.35,46.5,95.82,1.10,63.9,17.5,"MULTIPOLYGON (((664467.550 2716035.681, 664469...",0.000000,467
5,2044.0,natural_Soil,9.866122,-2.731568,7.134554,43.415371,no,23.86,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((664419.670 2716057.722, 664418...",34.996097,197
6,563.0,natural_Soil,51.801119,-14.823175,36.977944,413.317870,no,33.39,154.8,318.70,2.20,428.6,273.8,"MULTIPOLYGON (((664434.305 2716040.607, 664435...",0.000000,1909


In [27]:
# the numbers differ significantly from DD output.